In [ ]:
import os
import sys

from google.colab import drive
drive.mount('/content/gdrive')
project_dir = '/content/gdrive/MyDrive/Schism/'
sys.path.append(os.path.join(project_dir, "code/SCHISM/"))


Mounted at /content/gdrive


In [ ]:
import os
import numpy as np
import tifffile
from tqdm import tqdm

def compute_mean_std_online(image_folder):
    """
    Computes mean and standard deviation for all TIFF images in a folder using an online method.

    Args:
        image_folder (str): Path to the folder containing TIFF images.

    Returns:
        dict: Mean and standard deviation per RGB channel.
    """
    count = 0
    mean = np.zeros(3, dtype=np.float64)
    M2 = np.zeros(3, dtype=np.float64)  # Sum of squared differences from mean

    # List all TIFF images in the folder
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith((".tif", ".tiff"))])

    # Iterate through all TIFF images with a progress bar
    for filename in tqdm(image_files, desc="Processing Images"):
        img_path = os.path.join(image_folder, filename)
        img = tifffile.imread(img_path)

        # Ensure image is in (H, W, C) format
        if img.ndim == 2:  # Convert grayscale to pseudo-RGB
            img = np.stack([img] * 3, axis=-1)
        elif img.ndim == 3 and img.shape[0] == 3:  # C x H x W format
            img = np.transpose(img, (1, 2, 0))

        # Normalize to [0,1] if in uint8 format
        if img.dtype == np.uint8:
            img = img.astype(np.float32) / 255.0

        # Reshape image to a (N, 3) array where N = H * W
        pixels = img.reshape(-1, 3)

        # Update running mean and variance using Welford's method
        for pixel in pixels:
            count += 1
            delta = pixel - mean
            mean += delta / count
            M2 += delta * (pixel - mean)

    std = np.sqrt(M2 / (count - 1)) if count > 1 else np.zeros(3)

    return {
        "channel_means": mean.tolist(),
        "channel_stds": std.tolist()
    }

# Example usage
image_folder = os.path.join(project_dir, 'data', 'EO', 'EO3', 'images')
stats = compute_mean_std_online(image_folder)
print(stats)


Processing Images: 100%|██████████| 6920/6920 [47:41<00:00,  2.42it/s]

{'channel_means': [0.3573890401200933, 0.37838531579974277, 0.3453054259247108], 'channel_stds': [0.2322722929529305, 0.2275715934784253, 0.22415137921386658]}
